In [1]:
import spacy 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import datetime as dt 
import spacy
from collections import Counter 
import json
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words

In [2]:
submissions = pd.read_json("/Users/ritikabrahmadesam/Downloads/opiates_submissions2.json")

### This analysis aims to track the frequencies of certain words over time

In [3]:
#create a column with real datetimes
submissions['created_utc'] = submissions['created_utc'].apply(lambda date: dt.datetime.fromtimestamp(date))
submissions = submissions.sort_values(by='created_utc', ascending=True)

In [4]:
#remove the stop words and punctionationfrom a line of text
def remove_stops(text):
    nlp = spacy.load("en_core_web_sm")
    words = [word.lower() for word in text.split() if word.lower() not in sw_spacy]
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    new_text = " ".join(words)
    for elem in punc:
        new_text = new_text.replace(elem, "")
    return new_text


In [37]:
df_2012 = submissions[submissions['created_utc'].dt.year==2011]

In [38]:
df_2012 = submissions[submissions['created_utc'].dt.year==2012]
curr_df = {"january": df_2012[df_2012['created_utc'].dt.month==1]}
             #"febuary": submissions[submissions['created_utc'].dt.month==2]}
            #  "march": submissions[submissions['created_utc'].dt.month==3],
            #  "april": submissions[submissions['created_utc'].dt.month==4],
            #  "may": submissions[submissions['created_utc'].dt.month==5],
            #  "june": submissions[submissions['created_utc'].dt.month==6],
            #  "august": submissions[submissions['created_utc'].dt.month==8],
            #  "september": submissions[submissions['created_utc'].dt.month==9],
            #  "october": submissions[submissions['created_utc'].dt.month==10],
            #  "november": submissions[submissions['created_utc'].dt.month==11],
            #  "december": submissions[submissions['created_utc'].dt.month==12]



In [39]:
curr_df["january"]

,author,author_flair_css_class,author_flair_text,created_utc,domain,full_link,gilded,id,is_self,media_embed,...,thumbnail,title,url,user_reports,created,author_created_utc,author_fullname,edited,media,distinguished
6699,pitchthisbitch,,Feels Goodman,2012-01-01 03:13:06,self.opiates,https://www.reddit.com/r/opiates/comments/nyfc...,NaN,nyfcw,True,{},...,self,DAE feel like they're living a double life on ...,http://www.reddit.com/r/opiates/comments/nyfcw...,None,1325419986,NaN,None,NaN,None,None
6698,Xaelon,None,The Counselor,2012-01-01 12:54:04,self.opiates,https://www.reddit.com/r/opiates/comments/nyps...,NaN,nypso,True,{},...,self,Tramadol and drug tests,http://www.reddit.com/r/opiates/comments/nypso...,None,1325454844,1.323184e+09,t2_6dw9o,1.0,None,None
6697,coulrophobiaa,,Heroin,2012-01-01 14:10:10,imgur.com,https://www.reddit.com/r/opiates/comments/nysc...,NaN,nysc6,False,{},...,http://a.thumbs.redditmedia.com/QdLaySdtMSNTfT...,My one true love.,http://imgur.com/mwTpP,None,1325459410,1.320908e+09,t2_67tqj,NaN,None,None
6696,[deleted],None,None,2012-01-01 14:23:40,self.opiates,https://www.reddit.com/r/opiates/comments/nysu...,NaN,nysu4,True,{},...,default,Exalgo 24h ER,http://www.reddit.com/r/opiates/comments/nysu4...,None,1325460220,NaN,None,NaN,None,None
6695,[deleted],None,None,2012-01-01 22:04:34,reddit.com,https://www.reddit.com/r/opiates/comments/nzah...,NaN,nzah1,False,{},...,default,Any advice on quitting for a 4 year addict?,http://www.reddit.com/r/opiates/submit,None,1325487874,NaN,None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6419,[deleted],None,None,2012-01-31 19:28:36,self.opiates,https://www.reddit.com/r/opiates/comments/p5cf...,NaN,p5cf3,True,{},...,default,Tolerance Question,http://www.reddit.com/r/opiates/comments/p5cf3...,None,1328070516,NaN,None,NaN,None,None
6418,DanTheMan021,,Oxyphine,2012-01-31 20:34:28,self.opiates,https://www.reddit.com/r/opiates/comments/p5fm...,NaN,p5fmg,True,{},...,self,Good movies while nodding.,http://www.reddit.com/r/opiates/comments/p5fmg...,None,1328074468,1.327717e+09,t2_6ru4t,NaN,None,None
6417,[deleted],None,None,2012-01-31 20:48:08,self.opiates,https://www.reddit.com/r/opiates/comments/p5ga...,NaN,p5gan,True,{},...,default,Opana Potentiation,http://www.reddit.com/r/opiates/comments/p5gan...,None,1328075288,NaN,None,NaN,None,None
6416,BoyBreakSky,None,None,2012-01-31 21:10:50,self.opiates,https://www.reddit.com/r/opiates/comments/p5he...,NaN,p5he9,True,{},...,self,100mg Morphine. Best way to take it?,http://www.reddit.com/r/opiates/comments/p5he9...,None,1328076650,1.318782e+09,t2_61xyu,NaN,None,None


In [40]:
#clean the text by removing stop words, punctuation, and empty entries
#for month in month_dfs.keys():
curr_month = curr_df["january"]
text = curr_month['selftext'].to_list()
text = [remove_stops(t) for t in text]
while("" in text):
    text.remove("")
text = " ".join(text)
text = text.split()
print(text)
frequent = Counter(text) 
most_occur = frequent.most_common(50) 
json_format = json.dumps(dict(most_occur))
print(json_format)


['recently', 'account', 'decided', 'posting', 'detail', 'drug', 'life', 'public', 'forum', 'people', 'recognize', 'posting', 'weeks', 'realized', 'im', 'living', 'way', 'online', 'real', 'life', 'dont', 'know', 'feel', 'bad', 'thought', 'someone', 'feel', 'way', 'bad', 'recreational', 'drug', 'useself', 'medication', 'arent', 'socially', 'acceptable', 'possible', 'test', 'for', 'tested', 'extended', 'panels', 'all', 'edit', 'dosages', 'ultram', 'nice', 'warm', 'opiate', 'buzz', 'took', 'tramadol', 'maybe', 'seven', 'years', 'ago', 'dont', 'remember', 'anything', 'hello', 'time', 'posting', 'ive', 'browsing', 'while', 'decided', 'new', 'account', 'join', 'discussions', 'ask', 'questions', 'new', 'formula', 'hydromorphone', 'time', 'release', 'its', '24', 'hours', 'release', 'exalgo', 'im', 'start', 'taking', 'monday', 'night', 'wait', 'dropped', '30mcgh', 'butrans', 'patches', 'friday', 'ive', 'taking', 'month', 'condition', 'year', 'half', 'helping', 'all', 'doctor', 'switched', 'exalg

In [42]:
with open("january_2012.json", "w") as outfile:
    outfile.write(json_format)